# **ETL**

## **Librerías**

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sqlalchemy import create_engine

## **Datos - Cargue**

In [47]:
df_trades = pd.read_csv('./Datos/operaciones.csv', sep=';')
df_countries = pd.read_json('./Datos/paises.json')
df_codes = pd.read_csv('./Datos/productos.csv')

In [48]:
df_parents = df_codes.query('Level == 2').copy()

## **Limpieza y Transformación**

In [49]:
df_codes = df_codes[df_codes['Code_comm'].notnull()]

In [50]:
def clean_code(text):
    text = str(text)
    parent_code = None
    
    if len(text) == 11:
        code = text[:5]
        parent_code = text[:1]
    else:
        code = text[:6]
        parent_code = text[:2]    
    
    try:
        #parent = df_parents.query(f'Code_comm == {parent_code}')['Description'].values[0]
        parent = df_parents[df_parents['Code_comm']==parent_code]['Description'].values[0]
    except:
        parent = None
    
    return (code, parent)

In [51]:
df_codes[['clean_code', 'parent_description']] = df_codes.apply(
    lambda x: clean_code(x['Code']), 
    axis=1, 
    result_type='expand'
)

In [52]:
df_codes = df_codes[df_codes['clean_code'].notnull()]

In [54]:
df_codes[['clean_code', 'Description', 'parent_description']]

,clean_code,Description,parent_description
1,10011,LIVE ANIMALS; ANIMAL PRODUCTS,LIVE ANIMALS
2,10021,LIVE ANIMALS,LIVE ANIMALS
3,10100,"Live horses, asses, mules and hinnies",LIVE ANIMALS
5,10121,Pure-bred breeding horses,LIVE ANIMALS
6,10129,Live horses (excl. pure-bred for breeding),LIVE ANIMALS
...,...,...,...
7432,970200,"Original engravings, prints and lithographs","WORKS OF ART, COLLECTORS' PIECES AND ANTIQUES"
7433,970300,"Original sculptures and statuary, in any material","WORKS OF ART, COLLECTORS' PIECES AND ANTIQUES"
7434,970400,"Postage or revenue stamps, stamp-postmarks, fi...","WORKS OF ART, COLLECTORS' PIECES AND ANTIQUES"
7435,970500,Collections and collector's pieces of zoologic...,"WORKS OF ART, COLLECTORS' PIECES AND ANTIQUES"
